In [5]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

stop_words = list(set(stopwords.words('english')))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[{}0-9]'.format(string.punctuation), ' ', text)
    text=re.sub(r'[^A-Za-z0-9 ]+', ' ', text)
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    text = [WordNetLemmatizer().lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

In [6]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import swifter

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df=pd.DataFrame({"content":newsgroups["data"]})

df=df.sample(1000) #for a sample of 1000 documents
df["content"]=df["content"].swifter.apply(lambda x: preprocess_text(x))
df['content_length'] = df['content'].str.len()

df = df[df['content_length'] > 100]
df = df[df['content_length'] < 2000]

df=df[["content"]].reset_index(drop=True).reset_index().rename(columns={"index":"id"})
documents=df.content.to_list()

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
from bertopic import BERTopic


topic_model = BERTopic()
topics, probs = topic_model.fit_transform(documents)
#topic_model.save("./results/bertopic")

In [8]:
topic_model.get_topics()

{0: [('one', 0.029749398782808755),
  ('would', 0.02711250102041625),
  ('know', 0.02341354368872541),
  ('like', 0.022782732054443523),
  ('get', 0.020933646246006445),
  ('time', 0.01858034662116088),
  ('also', 0.0177861362507444),
  ('people', 0.017749437932748604),
  ('use', 0.016017927973628568),
  ('make', 0.015686517071045052)],
 1: [('game', 0.0641031055784542),
  ('team', 0.05481182112879568),
  ('cub', 0.05439483381059699),
  ('year', 0.04378709518978986),
  ('suck', 0.04028388861654908),
  ('player', 0.037384460864833505),
  ('time', 0.03434030459160824),
  ('hockey', 0.03320755164426929),
  ('pitcher', 0.029099836679088623),
  ('season', 0.02656941609829831)]}